In [ ]:
# Cell 1
# -*- coding: utf-8 -*-
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, label_binarize
import numpy as np
import torch.backends.cudnn as cudnn
from datetime import datetime
import matplotlib.pyplot as plt  # NEW: for line charts

# Global settings / reproducibility
cudnn.benchmark = True
torch.cuda.empty_cache()
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Root path to UCRArchive_2018
ROOT = r"D:\2025暑期科研\UCRArchive_2018\UCRArchive_2018"

In [ ]:
# Cell 2
def clean_and_pad_timeseries(raw_2d, min_len=8, cap_len=None, pad_value=0.0,
                             per_sample_standardize=True, fixed_len=None):
    """
    Clean time series with tail NaNs, z-score per-sample (optional), and pad/clip.

    Priority of output length:
      1) fixed_len: if not None, output length = fixed_len (force)
      2) cap_len:   if not None, output length = min(max_real_len, cap_len)
      3) otherwise, output length = max_real_len of this input batch
    """
    import numpy as np

    N, T = raw_2d.shape
    rows, keep_idx, real_lens = [], [], []

    for i in range(N):
        row = raw_2d[i]
        valid_vals = row[~np.isnan(row)]
        L = valid_vals.shape[0]
        if L < min_len:
            continue
        if per_sample_standardize:
            mu = valid_vals.mean()
            sigma = valid_vals.std()
            valid_vals = (valid_vals - mu) / (sigma if sigma > 0 else 1.0)
        rows.append(valid_vals); keep_idx.append(i); real_lens.append(L)

    if len(rows) == 0:
        raise ValueError("All samples filtered out. Lower min_len if needed.")

    max_real_len = max(real_lens)
    if fixed_len is not None:
        target_len = int(fixed_len)
    elif cap_len is not None:
        target_len = min(max_real_len, cap_len)
    else:
        target_len = max_real_len

    out = []
    for arr in rows:
        if arr.shape[0] >= target_len:
            arr = arr[:target_len]
        else:
            arr = np.pad(arr, (0, target_len - arr.shape[0]), constant_values=pad_value)
        out.append(arr)

    X = np.stack(out, axis=0).astype("float32")
    return X, np.array(keep_idx, dtype=np.int64), np.array(real_lens, dtype=np.int64)

In [ ]:
# Cell 3
class TwoTowerTransformer(nn.Module):
    """
    Two-tower Transformer:
      - Tower 1: time series tokens [B, T, 1] -> embed -> transformer
      - Tower 2: Aout vector [B, F] as a single token -> embed -> transformer
      - Concat tokens -> final transformer -> flatten -> FC for multi-class logits
    """
    def __init__(self, input_dim1, input_dim2,
                 hidden_dim1, hidden_dim2, hidden_dim3,
                 num_heads, num_layers, num_classes,
                 seq_len1, seq_len2):
        super().__init__()
        # To keep it simple we force equal hidden dims and divisibility by nhead
        assert hidden_dim1 == hidden_dim2 == hidden_dim3, "hidden dims must be equal in this version."
        for h in (hidden_dim1, hidden_dim2, hidden_dim3):
            assert h % num_heads == 0, "hidden_dim must be divisible by num_heads"

        self.embedding1 = nn.Linear(input_dim1, hidden_dim1)
        self.embedding2 = nn.Linear(input_dim2, hidden_dim2)
        self.relu = nn.ReLU()

        self.transformer1 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim1, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.transformer2 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim2, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.final_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim3, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.dropout = nn.Dropout(0.3)
        self.seq_len1 = seq_len1
        self.seq_len2 = 1  # treat Aout as a single token
        fc_in = hidden_dim3 * (seq_len1 + self.seq_len2)
        self.fc = nn.Linear(fc_in, num_classes)

    def forward(self, x1, x2):
        # x1: [B, T, input_dim1], x2: [B, F] or [B, 1, F]
        if x1.dim() == 2:
            x1 = x1.unsqueeze(-1)
        x1 = self.relu(self.embedding1(x1))
        x1 = self.transformer1(x1)

        if x2.dim() == 3:
            assert x2.size(1) == 1, "Expect x2 with L2=1 if 3D"
            x2 = x2.squeeze(1)
        x2 = self.relu(self.embedding2(x2))
        x2 = x2.unsqueeze(1)
        x2 = self.transformer2(x2)

        x = torch.cat((x1, x2), dim=1)
        x = self.final_transformer(x)
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)


class VisitDataset(Dataset):
    """Simple tensor dataset for (visit time series, aout features, one-hot labels)."""
    def __init__(self, visit_x, aout_x, y):
        self.visit_x = visit_x.astype("float32")
        self.aout_x  = aout_x.astype("float32")
        self.y       = y.astype("float32")
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return self.visit_x[idx], self.aout_x[idx], self.y[idx]

In [ ]:
# Cell 4
# ===== Backbones: Informer / FEDformer (Encoder-only Classifiers) =====
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class LearnedPositionalEncoding(nn.Module):
    def __init__(self, seq_len: int, d_model: int, dropout: float = 0.0):
        super().__init__()
        self.pe = nn.Parameter(torch.zeros(1, seq_len, d_model))
        nn.init.trunc_normal_(self.pe, std=0.02)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1), :])

class TokenEmbedding(nn.Module):
    def __init__(self, input_dim: int, d_model: int):
        super().__init__()
        self.proj = nn.Linear(input_dim, d_model)
    def forward(self, x):  # x: [B, L, C]
        return self.proj(x)

class InformerEncoderLayer(nn.Module):
    def __init__(self, d_model=256, n_heads=4, d_ff=512, dropout=0.1, attn_dropout=0.1, activation='gelu'):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=attn_dropout, batch_first=True)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU() if activation.lower()=='gelu' else nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)
    def forward(self, x, attn_mask=None, key_padding_mask=None):
        attn_out, _ = self.self_attn(x, x, x, attn_mask=attn_mask, key_padding_mask=key_padding_mask, need_weights=False)
        x = self.norm1(x + self.dropout1(attn_out))
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout2(ff_out))
        return x

class DistilConv1D(nn.Module):
    def __init__(self, d_model, dropout=0.0):
        super().__init__()
        self.conv = nn.Conv1d(d_model, d_model, kernel_size=3, stride=2, padding=1)
        self.norm = nn.BatchNorm1d(d_model)
        self.act = nn.ELU()
        self.drop = nn.Dropout(dropout)
    def forward(self, x):  # [B, L, D]
        x = x.transpose(1, 2)
        x = self.conv(x)
        x = self.norm(x)
        x = self.act(x)
        x = self.drop(x)
        return x.transpose(1, 2)

class InformerEncoder(nn.Module):
    def __init__(self, d_model=256, n_heads=4, e_layers=2, d_ff=512, dropout=0.1, distil=True, attn_dropout=0.1, activation='gelu'):
        super().__init__()
        self.layers = nn.ModuleList([
            InformerEncoderLayer(d_model, n_heads, d_ff, dropout, attn_dropout, activation)
            for _ in range(e_layers)
        ])
        self.distil_layers = nn.ModuleList([DistilConv1D(d_model, dropout=dropout) for _ in range(e_layers-1)]) if distil and e_layers>1 else None
    def forward(self, x):  # [B, L, D]
        if self.distil_layers is None:
            for enc in self.layers: x = enc(x)
            return x
        for i,enc in enumerate(self.layers):
            x = enc(x)
            if i < len(self.layers)-1: x = self.distil_layers[i](x)
        return x

# ====== REPLACE in Cell 4: FourierBlock & FEDEncoderLayer ======

class FourierAttentionLike(nn.Module):
    """
    '更贴近官方'的频域块：
    - rFFT 得到频域
    - 选能量最大的 top-k 频率（按 batch 的平均能量统计，不会引入数据泄漏）
    - 对选中的频率施加【可学习的复权重】(幅度 + 相位)
    - iFFT 回时域
    仍然是 Encoder 内的一个子层，用作“频域相关性”的近似。
    """
    def __init__(self, d_model: int, k_ratio: float = 0.25):
        super().__init__()
        self.k_ratio = k_ratio
        # 可学习的幅度与相位（简化：对所有通道共享一组权重；也可换成 per-channel）
        self.scale = nn.Parameter(torch.ones(1))          # 幅度缩放
        self.phase = nn.Parameter(torch.zeros(1))         # 相位偏移（弧度）
        self.proj_in  = nn.Identity()                     # 你也可以放个 1x1 线性层
        self.proj_out = nn.Identity()

    def forward(self, x):  # x: [B, L, D]
        B, L, D = x.shape
        x = self.proj_in(x)

        # FFT：对时间维做 rFFT -> 形状 [B, F, D]，F = L//2 + 1
        Xf = torch.fft.rfft(x, dim=1)

        # 频率能量（幅度平方），取 batch&channel 平均，得到每个频率的能量
        energy = (Xf.real**2 + Xf.imag**2).mean(dim=(0, 2))  # [F]
        F = Xf.size(1)
        k = max(1, int(F * self.k_ratio))

        # top-k 频率索引（按能量挑最重要的频率）
        topk_idx = torch.topk(energy, k=k, largest=True, sorted=False).indices

        # 构造一个 mask 仅保留 top-k 频率
        mask = torch.zeros(F, device=Xf.device, dtype=Xf.dtype)
        mask[topk_idx] = 1.0
        mask = mask.view(1, F, 1)  # broadcast 到 [B, F, D]

        # 对保留的频率应用可学习复权重：scale * exp(j*phase)
        complex_weight = self.scale * torch.complex(
            torch.cos(self.phase), torch.sin(self.phase)
        )  # 标量复数
        Xf = Xf * mask * complex_weight

        # 反变换回时域
        y = torch.fft.irfft(Xf, n=L, dim=1)
        y = self.proj_out(y)
        return y


class FEDEncoderLayer(nn.Module):
    def __init__(self, d_model=256, d_ff=512, dropout=0.1, activation='gelu', k_ratio=0.25):
        super().__init__()
        # 用上面“更贴近官方”的频域相关性近似块
        self.fourier = FourierAttentionLike(d_model=d_model, k_ratio=k_ratio)

        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)

        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU() if activation.lower()=='gelu' else nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        # 频域相关性（残差）
        z = self.fourier(x)
        x = self.norm1(x + self.dropout1(z))
        # FFN（残差）
        z2 = self.ff(x)
        x = self.norm2(x + self.dropout2(z2))
        return x

class FEDformerEncoder(nn.Module):
    def __init__(self, d_model=256, e_layers=2, d_ff=512, dropout=0.1, activation='gelu', k_ratio=0.25):
        super().__init__()
        self.layers = nn.ModuleList([FEDEncoderLayer(d_model, d_ff, dropout, activation, k_ratio) for _ in range(e_layers)])
    def forward(self, x):
        for enc in self.layers: x = enc(x)
        return x

class TimePoolClassifierHead(nn.Module):
    def __init__(self, d_model: int, num_classes: int, pool: str = "mean", dropout: float = 0.1):
        super().__init__()
        assert pool in ("mean","max","cls")
        self.pool = pool
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, num_classes)
        self.cls_token = nn.Parameter(torch.zeros(1,1,d_model)) if pool=="cls" else None
        if self.cls_token is not None: nn.init.trunc_normal_(self.cls_token, std=0.02)
    def forward(self, x):  # [B, L, D]
        if self.pool=="mean":
            h = x.mean(dim=1)
        elif self.pool=="max":
            h,_ = x.max(dim=1)
        else:
            h = x[:,0,:]
        h = self.dropout(h)
        return self.fc(h)

class InformerEncoderClassifier(nn.Module):
    """Drop-in 替换 TwoTower；forward(x1,x2) 忽略 x2；x1=[B,L,1]"""
    def __init__(self, input_dim:int, num_classes:int, seq_len:int,
                 d_model=256, n_heads=4, e_layers=2, d_ff=512,
                 dropout=0.1, attn_dropout=0.1, distil=True, activation='gelu', pool="mean"):
        super().__init__()
        self.token = TokenEmbedding(input_dim, d_model)
        self.pos = LearnedPositionalEncoding(seq_len, d_model, dropout)
        self.encoder = InformerEncoder(d_model, n_heads, e_layers, d_ff, dropout, distil, attn_dropout, activation)
        self.head = TimePoolClassifierHead(d_model, num_classes, pool, dropout)
    def forward(self, x1, x2=None):
        if x1.dim()==2: x1 = x1.unsqueeze(-1)
        x = self.token(x1); x = self.pos(x); x = self.encoder(x)
        return self.head(x)

class FEDformerEncoderClassifier(nn.Module):
    """Drop-in 替换 TwoTower；forward(x1,x2) 忽略 x2；x1=[B,L,1]"""
    def __init__(self, input_dim:int, num_classes:int, seq_len:int,
                 d_model=256, e_layers=2, d_ff=512, dropout=0.1, activation='gelu', k_ratio=0.25, pool="mean"):
        super().__init__()
        self.token = TokenEmbedding(input_dim, d_model)
        self.pos = LearnedPositionalEncoding(seq_len, d_model, dropout)
        self.encoder = FEDformerEncoder(d_model, e_layers, d_ff, dropout, activation, k_ratio)
        self.head = TimePoolClassifierHead(d_model, num_classes, pool, dropout)
    def forward(self, x1, x2=None):
        if x1.dim()==2: x1 = x1.unsqueeze(-1)
        x = self.token(x1); x = self.pos(x); x = self.encoder(x)
        return self.head(x)
    
# === TS2Vec helpers: 自监督预训练 + 线性探针 ===
# 依赖：请把官方仓库的 ts2vec.py、models/*.py、utils/*.py 放到你的工程目录里（或同级子目录并做好 import）
from ts2vec import TS2Vec
import torch.nn as nn
import torch.nn.functional as F

def ts2vec_linear_probe(
    visit_tr, visit_te,             # np array: [N, L, 1] （左塔）
    Y_tr, Y_te,                     # one-hot labels
    device,
    batch_size=8,
    pretrain_iters=None,            # 若为 None，走官方默认（<=1e5元素 200iters，否则 600）
    pretrain_epochs=None,           # 二选一，给 iters 或 epochs
    encoder_hidden=64,
    encoder_depth=10,
    proj_dim=320,
    lr=1e-4,                        # 为公平，用你全局 lr；若要更贴近官方，可把这项设为1e-3
    head_epochs=100,                # 用与你主干一致的 num_epochs
    patience=15                     # 只对 head(监督)训练早停；自监督阶段不早停
):
    assert visit_tr.ndim == 3 and visit_tr.shape[-1] == 1
    assert visit_te.ndim == 3 and visit_te.shape[-1] == 1
    # --- 1) 自监督预训练（只用 train） ---
    ts2 = TS2Vec(
        input_dims=1,
        output_dims=proj_dim,
        hidden_dims=encoder_hidden,
        depth=encoder_depth,
        device=device,
        lr=lr,                 # 如想更贴近官方，可单独设 lr=1e-3
        batch_size=batch_size,
        max_train_length=None, # 我们已做同长，不再额外切窗
        temporal_unit=0
    )
    # 官方 fit 支持 n_iters 或 n_epochs；给一个即可
    loss_log = ts2.fit(
        train_data=visit_tr, 
        n_iters=pretrain_iters, 
        n_epochs=pretrain_epochs, 
        verbose=False
    )

    # --- 2) 编码为全序列表征（train/test 各得到一个向量表示） ---
    # encoding_window='full_series' 会做全序列 max-pool，输出 [N, 1, proj_dim] -> squeeze 到 [N, proj_dim]
    z_tr = ts2.encode(visit_tr, encoding_window='full_series')  # -> [Ntr, D]
    z_te = ts2.encode(visit_te, encoding_window='full_series')  # -> [Nte, D]

    # --- 3) 线性探针分类头（监督训练，与你现有超参/早停一致） ---
    num_classes = Y_tr.shape[1]
    z_tr_t = torch.from_numpy(z_tr).to(device=device, dtype=torch.float32)
    z_te_t = torch.from_numpy(z_te).to(device=device, dtype=torch.float32)
    y_tr_t = torch.from_numpy(Y_tr).to(device=device, dtype=torch.float32)
    y_te_t = torch.from_numpy(Y_te).to(device=device, dtype=torch.float32)

    head = nn.Linear(proj_dim, num_classes).to(device)
    optim_head = torch.optim.Adam(head.parameters(), lr=lr)
    sched_head = torch.optim.lr_scheduler.CosineAnnealingLR(optim_head, T_max=50, eta_min=0.0)
    criterion = nn.BCEWithLogitsLoss()

    # 简单的张量版 mini-batch loader
    def iter_minibatch(X, Y, bs):
        n = X.size(0)
        idx = torch.randperm(n, device=X.device)
        for i in range(0, n, bs):
            j = idx[i:i+bs]
            yield X[j], Y[j]

    class _ES:
        def __init__(self, patience=15, delta=0.0):
            self.pat = patience; self.delta=delta
            self.best=None; self.count=0; self.stop=False
        def step(self, loss):
            if self.best is None: self.best = loss; return False
            if loss > self.best - self.delta:
                self.count += 1
                if self.count >= self.pat: self.stop=True
            else:
                self.best = loss; self.count=0
            return self.stop
    es = _ES(patience=patience)

    # 监督阶段：只看 TrainLoss 早停（与你现在管线一致）
    for ep in range(head_epochs):
        head.train()
        tot=0.0; steps=0
        for xb, yb in iter_minibatch(z_tr_t, y_tr_t, batch_size):
            optim_head.zero_grad()
            logits = head(xb)
            loss = criterion(logits, yb)
            loss.backward(); optim_head.step()
            tot += loss.item(); steps+=1
        sched_head.step()
        avg_train_loss = tot / max(1,steps)
        if es.step(avg_train_loss): break

    # --- 4) 最终 Test 一次性评估 ---
    head.eval()
    with torch.no_grad():
        logits = head(z_te_t).detach().cpu().numpy()
    import numpy as np
    from sklearn.metrics import roc_auc_score, accuracy_score
    y_true = y_te_t.detach().cpu().numpy()
    try:
        test_auc = roc_auc_score(y_true, logits, multi_class='ovr')
    except:
        test_auc = roc_auc_score(y_true, logits)
    y_pred = np.argmax(logits, axis=1)
    y_cls  = np.argmax(y_true,  axis=1)
    test_acc = accuracy_score(y_cls, y_pred)
    return float(test_auc), float(test_acc), int(y_true.shape[0])

In [ ]:
# Cell 5
# ==== TS2Vec 线性探针：每个 epoch 记录一次 test BCE loss（用于画曲线）====
from ts2vec import TS2Vec
import torch, torch.nn as nn, torch.optim as optim
import numpy as np

def ts2vec_linear_probe(
    visit_tr, visit_te,            # np: [N, L, 1]
    Y_tr, Y_te,                    # one-hot
    device='cuda:0',
    batch_size=8,
    pretrain_iters=None,           # 不给就用官方默认
    pretrain_epochs=None,
    encoder_hidden=64,
    encoder_depth=10,
    proj_dim=320,
    lr=1e-4,                       # 与其它骨干统一
    head_epochs=60,                # 线性头训练总 epoch（建议与你收集器 NUM_EPOCHS 对齐）
    patience=15,
    collect_test_curve=True,       # 记录 test loss 折线
    force_full_epochs=True         # True=无视早停，强制跑满 head_epochs
):
    # 1) 自监督预训练（仅用 train）
    ts2 = TS2Vec(
        input_dims=1,
        output_dims=proj_dim,
        hidden_dims=encoder_hidden,
        depth=encoder_depth,
        device=device,
        lr=lr,
        batch_size=batch_size,
        max_train_length=None,
        temporal_unit=0
    )
    _ = ts2.fit(
        train_data=visit_tr,
        n_iters=pretrain_iters,
        n_epochs=pretrain_epochs,
        verbose=False
    )

    # 2) 得到整序列表示（全序列池化）
    z_tr = ts2.encode(visit_tr, encoding_window='full_series')   # [Ntr, D]
    z_te = ts2.encode(visit_te, encoding_window='full_series')   # [Nte, D]

    # 3) 线性探针
    X_tr = torch.from_numpy(z_tr).to(torch.float32).to(device)
    X_te = torch.from_numpy(z_te).to(torch.float32).to(device)
    y_tr = torch.from_numpy(Y_tr).to(torch.float32).to(device)
    y_te = torch.from_numpy(Y_te).to(torch.float32).to(device)

    head = nn.Linear(X_tr.shape[1], y_tr.shape[1]).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(head.parameters(), lr=lr)

    class EarlyStopping:
        def __init__(self, patience=15, delta=0.0):
            self.patience = patience; self.delta = delta
            self.counter = 0; self.best = None; self.stop = False
        def step(self, loss):
            if self.best is None: self.best = loss; return False
            if loss > self.best - self.delta:
                self.counter += 1
                if self.counter >= self.patience: self.stop = True
            else:
                self.best = loss; self.counter = 0
            return self.stop
    es = EarlyStopping(patience=patience)

    dl = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(X_tr, y_tr),
        batch_size=batch_size, shuffle=True
    )

    test_loss_hist = []

    for epoch in range(head_epochs):
        # ----- train -----
        head.train()
        total = 0.0
        for xb, yb in dl:
            optimizer.zero_grad()
            logits = head(xb)
            loss = criterion(logits, yb)
            loss.backward(); optimizer.step()
            total += loss.item()

        # ----- per-epoch test loss -----
        if collect_test_curve:
            head.eval()
            with torch.no_grad():
                te_logits = head(X_te)
                te_loss = criterion(te_logits, y_te).item()
            test_loss_hist.append(float(te_loss))

        # 仅在不强制满 epoch 时才早停
        if (not force_full_epochs) and es.step(total / max(1, len(dl))):
            break

    # 最终一次性测试指标
    head.eval()
    with torch.no_grad():
        logits = head(X_te).detach().cpu().numpy()
        labs   = y_te.detach().cpu().numpy()
    from sklearn.metrics import accuracy_score, roc_auc_score
    try:  auc = roc_auc_score(labs, logits, multi_class='ovr')
    except: auc = roc_auc_score(labs, logits)
    pred = logits.argmax(1); true = labs.argmax(1)
    acc = accuracy_score(true, pred)

    return float(auc), float(acc), int(labs.shape[0]), test_loss_hist


In [ ]:
# Cell 6
def run_one_dataset(dataset_dir, dataset_name,
                    device='cuda:0',
                    batch_size=8, num_epochs=60, lr=1e-4,
                    cap_len=None, patience=15,
                    verbose=False, plot_curves=False,
                    backbone='ts2vec',              # 这里默认 ts2vec；按需改
                    # 给收集器用的三个参数
                    return_epoch_losses=False,       # True 时返回 test-loss 序列
                    force_full_epochs=True,          # True=强制跑满 num_epochs
                    collect_test_curve=True):
    import os, numpy as np, pandas as pd
    import torch, torch.nn as nn, torch.optim as optim
    from torch.utils.data import DataLoader
    from torch.optim.lr_scheduler import CosineAnnealingLR
    from sklearn.preprocessing import StandardScaler, label_binarize
    from sklearn.metrics import accuracy_score, roc_auc_score

    # ---------- 路径 ----------
    tsv_train_path = os.path.join(dataset_dir, f"{dataset_name}_TRAIN_cleaned.tsv")
    tsv_test_path  = os.path.join(dataset_dir, f"{dataset_name}_TEST_cleaned.tsv")
    aout_train_csv = os.path.join(dataset_dir, f"{dataset_name}_Aout_train_k2.csv")
    aout_test_csv  = os.path.join(dataset_dir, f"{dataset_name}_Aout_test_k2.csv")
    if not all(os.path.exists(p) for p in [tsv_train_path, tsv_test_path, aout_train_csv, aout_test_csv]):
        print(f"⚠ Skip {dataset_name}, missing files"); return None

    # ---------- 读取 ----------
    tsv_tr = pd.read_csv(tsv_train_path, sep="\t", header=None)
    tsv_te = pd.read_csv(tsv_test_path,  sep="\t", header=None)
    csv_tr = pd.read_csv(aout_train_csv, header=None)
    csv_te = pd.read_csv(aout_test_csv,  header=None)

    y_tr_raw = tsv_tr.iloc[:,0].values
    y_te_raw = tsv_te.iloc[:,0].values
    visit_tr_raw = tsv_tr.iloc[:,1:].values.astype("float32")
    visit_te_raw = tsv_te.iloc[:,1:].values.astype("float32")
    aout_tr_raw_all  = csv_tr.iloc[:,1:].values.astype("float32")
    aout_te_raw_all  = csv_te.iloc[:,1:].values.astype("float32")

    # ---------- Train 决定参考长度 ----------
    tmp_train_clean, keep_tr0, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=None
    )
    train_seq_len = tmp_train_clean.shape[1]

    # ---------- 固定同长清洗 ----------
    visit_tr_clean, keep_tr, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_tr = y_tr_raw[keep_tr]
    aout_tr_raw = aout_tr_raw_all[keep_tr]

    visit_te_clean, keep_te, _ = clean_and_pad_timeseries(
        visit_te_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_te = y_te_raw[keep_te]
    aout_te_raw = aout_te_raw_all[keep_te]

    # ---------- 右塔标准化（为与其它骨干对齐，TS2Vec不使用右塔，但此处保留流程一致性） ----------
    _ = StandardScaler().fit(aout_tr_raw)   # 不实际用到，保持接口兼容

    # ---------- One-hot 标签 ----------
    classes = sorted(np.unique(y_tr))
    Y_tr = label_binarize(y_tr, classes=classes).astype("float32")
    Y_te = label_binarize(y_te, classes=classes).astype("float32")

    # ---------- 左塔输入形状 [B, L, 1] ----------
    visit_tr = visit_tr_clean[:, :, None]
    visit_te = visit_te_clean[:, :, None]

    # ===== TS2Vec 分支：自监督 + 线性探针（在 Test 上逐 epoch 取 BCE loss）=====
    if backbone.lower() in ('ts2vec', 'ts2vec_lp'):
        device_t = torch.device(device if torch.cuda.is_available() else 'cpu')
        test_auc, test_acc, test_n, te_loss_hist = ts2vec_linear_probe(
            visit_tr=visit_tr, visit_te=visit_te,
            Y_tr=Y_tr, Y_te=Y_te,
            device=device_t,
            batch_size=batch_size,
            pretrain_iters=None, pretrain_epochs=None,
            encoder_hidden=64, encoder_depth=10, proj_dim=320,
            lr=lr,
            head_epochs=num_epochs,
            patience=patience,
            collect_test_curve=collect_test_curve,
            force_full_epochs=force_full_epochs
        )
        print(f"[{dataset_name} | ts2vec] TEST AUC={test_auc:.4f}, TEST ACC={test_acc:.4f}, n_samples={test_n}")
        if return_epoch_losses:
            return test_auc, test_acc, test_n, te_loss_hist
        else:
            return test_auc, test_acc, test_n

    # ===== 若误传了其它骨干，这里直接报错，避免混淆 =====
    raise ValueError("This run_one_dataset() 版本仅用于 TS2Vec 收集 test-loss。请使用对应骨干的 run_one_dataset()。")


In [ ]:
# Cell 7
def discover_datasets(root):
    """
    Discover dataset subfolders that contain all four required files:
      *_TRAIN_cleaned.tsv, *_TEST_cleaned.tsv, *_Aout_train_k2.csv, *_Aout_test_k2.csv
    Returns: a sorted list of dataset names (folder names).
    """
    names = []
    for name in sorted(os.listdir(root)):
        subdir = os.path.join(root, name)
        if not os.path.isdir(subdir):
            continue
        t_train = os.path.join(subdir, f"{name}_TRAIN_cleaned.tsv")
        t_test  = os.path.join(subdir, f"{name}_TEST_cleaned.tsv")
        a_train = os.path.join(subdir, f"{name}_Aout_train_k2.csv")
        a_test  = os.path.join(subdir, f"{name}_Aout_test_k2.csv")
        if all(os.path.exists(p) for p in [t_train, t_test, a_train, a_test]):
            names.append(name)
    return names

In [ ]:
# Cell 8
def run_all_datasets(root, device='cuda:0',
                     batch_size=8, num_epochs=100, lr=1e-4,
                     cap_len=None, verbose=False, plot_curves=True,
                     patience=15, backbone='twotower'):
    import os, pandas as pd
    from datetime import datetime

    dataset_names = discover_datasets(root)
    print(f"Found {len(dataset_names)} datasets:", dataset_names)

    rows = []
    for name in dataset_names:
        out = run_one_dataset(
            dataset_dir=os.path.join(root, name),
            dataset_name=name,
            device=device,
            batch_size=batch_size, num_epochs=num_epochs, lr=lr,
            cap_len=cap_len, patience=patience,
            verbose=verbose, plot_curves=plot_curves,
            backbone=backbone   # <-- 透传骨干
        )
        if out is None: continue
        test_auc, test_acc, test_n = out
        rows.append((name, test_auc, test_acc, test_n))

    if not rows:
        print("No dataset finished successfully."); return None

    df = pd.DataFrame(rows, columns=["dataset", "test_auc", "test_acc", "n_samples"])
    mean_auc = df["test_auc"].mean(); mean_acc = df["test_acc"].mean()
    w_auc = (df["test_auc"] * df["n_samples"]).sum() / df["n_samples"].sum()
    w_acc = (df["test_acc"] * df["n_samples"]).sum() / df["n_samples"].sum()

    print("\n========== Summary (OFFICIAL TEST) ==========")
    print(df.sort_values("dataset").to_string(index=False))
    print(f"\nSimple mean: AUC = {mean_auc:.4f}, ACC = {mean_acc:.4f}")
    print(f"Weighted (by samples): AUC = {w_auc:.4f}, ACC = {w_acc:.4f}")

    summary_dir = os.path.join(root, "_twotower_logs"); os.makedirs(summary_dir, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    tag = backbone.lower()
    df.to_csv(os.path.join(summary_dir, f"summary_TEST_{tag}_{ts}.csv"), index=False, encoding="utf-8")
    with open(os.path.join(summary_dir, f"summary_TEST_{tag}_{ts}.txt"), "w", encoding="utf-8") as f:
        f.write(df.sort_values("dataset").to_string(index=False))
        f.write(f"\n\nSimple mean: AUC={mean_auc:.6f}, ACC={mean_acc:.6f}\n")
        f.write(f"Weighted (by samples): AUC={w_auc:.6f}, ACC={w_acc:.6f}\n")

    return df, (mean_auc, mean_acc), (w_auc, w_acc)

In [ ]:
# Cell 9
# === BATCH COLLECTOR (TS2Vec): build logs/ts2vec_testloss.tsv ===
import os, numpy as np, pandas as pd

NUM_EPOCHS = 60
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)

dataset_names = discover_datasets(ROOT)
print(f"[TS2Vec] Discover {len(dataset_names)} datasets")

loss_table = pd.DataFrame(index=np.arange(1, NUM_EPOCHS+1),
                          columns=dataset_names, dtype=float)
loss_table.index.name = "epoch"

for ds in dataset_names:
    try:
        out = run_one_dataset(
            dataset_dir=os.path.join(ROOT, ds),
            dataset_name=ds,
            device='cuda:0',
            batch_size=8,
            num_epochs=NUM_EPOCHS,     # 线性头训练轮数=60
            lr=1e-4,
            cap_len=None,
            patience=15,
            verbose=False,
            plot_curves=False,
            backbone='ts2vec',         # 关键：走 ts2vec 分支
            return_epoch_losses=True,   # 要 test-loss 序列
            force_full_epochs=True,     # 强制跑满 60 个 epoch
            collect_test_curve=True     # 记录 test loss
        )
        if out is None:
            print(f"[SKIP] {ds} returned None")
            loss_table[ds] = np.nan
            continue

        # 期望 out: (test_auc, test_acc, test_n, test_loss_hist)
        _, _, _, epoch_losses = out

        if len(epoch_losses) != NUM_EPOCHS:
            print(f"[WARN] {ds}: got {len(epoch_losses)} epochs, expect {NUM_EPOCHS}. Pad/Crop.")
        epoch_losses = (list(epoch_losses) + [np.nan]*NUM_EPOCHS)[:NUM_EPOCHS]
        loss_table[ds] = epoch_losses

    except Exception as e:
        print(f"[ERROR] {ds}: {e}")
        loss_table[ds] = np.nan

out_path = os.path.join(LOG_DIR, "ts2vec_testloss.tsv")
loss_table.to_csv(out_path, sep="\t", float_format="%.6f")
print(f"[TS2Vec] Saved test-loss table to {out_path} with shape {loss_table.shape}")


[TS2Vec] Discover 125 datasets
[ACSF1 | ts2vec] TEST AUC=0.6981, TEST ACC=0.3300, n_samples=100
[Adiac | ts2vec] TEST AUC=0.6522, TEST ACC=0.0358, n_samples=391
[AllGestureWiimoteX | ts2vec] TEST AUC=0.6332, TEST ACC=0.2996, n_samples=681
[AllGestureWiimoteY | ts2vec] TEST AUC=0.6576, TEST ACC=0.3814, n_samples=645
[AllGestureWiimoteZ | ts2vec] TEST AUC=0.6446, TEST ACC=0.2803, n_samples=685
[ArrowHead | ts2vec] TEST AUC=0.6477, TEST ACC=0.5657, n_samples=175
[BME | ts2vec] TEST AUC=0.8771, TEST ACC=0.5133, n_samples=150
[Beef | ts2vec] TEST AUC=0.5097, TEST ACC=0.2000, n_samples=30
[BeetleFly | ts2vec] TEST AUC=0.9200, TEST ACC=1.0000, n_samples=20
[BirdChicken | ts2vec] TEST AUC=0.8800, TEST ACC=1.0000, n_samples=20
[CBF | ts2vec] TEST AUC=0.9548, TEST ACC=0.3322, n_samples=900
[Car | ts2vec] TEST AUC=0.5910, TEST ACC=0.2833, n_samples=60
[Chinatown | ts2vec] TEST AUC=0.7486, TEST ACC=1.0000, n_samples=343
[ChlorineConcentration | ts2vec] TEST AUC=0.5220, TEST ACC=0.5372, n_samples=3